In [1]:
import pandas as pd
import numpy as np
import datetime
import gtfstk as gt
import os, zipfile, requests, io
import multiprocessing as mp
from IPython.display import clear_output
%matplotlib inline

In [2]:
folder=r"N:\Planning - New File Structure\Performance\FY19 OTP\July_1_July_13_1_5"

In [3]:
zip_file_url=r"http://valleyregionaltransit.org/gtfs/VRT_Transit1.zip"
r = requests.get(zip_file_url)
z = zipfile.ZipFile(io.BytesIO(r.content))
z.extractall("gtfs")
gtfs_filename=os.path.basename(zip_file_url)
gtfs_filename=os.path.splitext(gtfs_filename)[0]
gtfs=gt.read_gtfs("gtfs",dist_units='mi')

In [4]:
feed=gtfs.stop_times.merge(gtfs.stops)
feed=feed.merge(gtfs.trips)
feed=feed.merge(gtfs.routes)
feed['Route Name']=feed['route_short_name']+" "+feed['route_long_name']
feed.head()

,trip_id,arrival_time,departure_time,stop_id,stop_sequence,stop_headsign,pickup_type,drop_off_type,shape_dist_traveled,stop_code,...,shape_id,agency_id,route_short_name,route_long_name,route_desc,route_type,route_url,route_color,route_text_color,Route Name
0,1_08:45_Weekday_82588,08:45:00,08:45:00,01019999,0,NaN,NaN,NaN,20.085339,NaN,...,shape_VRT_6526,VRT,1,Harris Ranch via Parkcenter,NaN,3,NaN,ffcd00,000000,1 Harris Ranch via Parkcenter
1,1_08:45_Weekday_82588,NaN,NaN,01010126,1,NaN,NaN,NaN,4331.011367,NaN,...,shape_VRT_6526,VRT,1,Harris Ranch via Parkcenter,NaN,3,NaN,ffcd00,000000,1 Harris Ranch via Parkcenter
2,1_08:45_Weekday_82588,NaN,NaN,01010154,2,NaN,NaN,NaN,6945.142323,NaN,...,shape_VRT_6526,VRT,1,Harris Ranch via Parkcenter,NaN,3,NaN,ffcd00,000000,1 Harris Ranch via Parkcenter
3,1_08:45_Weekday_82588,08:51:00,08:51:00,01010157,3,NaN,NaN,NaN,8648.791207,NaN,...,shape_VRT_6526,VRT,1,Harris Ranch via Parkcenter,NaN,3,NaN,ffcd00,000000,1 Harris Ranch via Parkcenter
4,1_08:45_Weekday_82588,NaN,NaN,01010638,4,NaN,NaN,NaN,11462.425138,NaN,...,shape_VRT_6526,VRT,1,Harris Ranch via Parkcenter,NaN,3,NaN,ffcd00,000000,1 Harris Ranch via Parkcenter


In [5]:
df=pd.DataFrame()

In [6]:
for i in os.listdir(folder):
    df=df.append(pd.read_csv(folder+"\\"+i,parse_dates=['ScheduledStopTime']))
    print('reading: {} ({})'.format(i,df.shape))
print('Done!')

reading: OTP_Detail_All_Routes.csv ((31073, 27))
Done!


In [7]:
df=df.merge(feed, left_on=['StopName','TripName','StopOrder','MasterRouteName'],right_on=['stop_name','trip_id','stop_sequence','route_long_name'],how='left')
df.head()

,VehicleName,RunName,MasterRouteName,SubRouteName,TripName,StopName,DriverName,TripDate,ScheduledTripStart,ScheduledTripEnd,...,shape_id,agency_id,route_short_name,route_long_name,route_desc,route_type,route_url,route_color,route_text_color,Route Name
0,NaN,NaN,Broadway,NaN,2_05:35_Weekday_82661,Boise Airport,NaN,7/1/2019 12:00:00 AM,7/1/2019 5:35:00 AM,7/1/2019 6:06:00 AM,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,Broadway,NaN,2_05:35_Weekday_82661,Federal Way @ Fred Meyer SWC,NaN,7/1/2019 12:00:00 AM,7/1/2019 5:35:00 AM,7/1/2019 6:06:00 AM,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,Broadway,NaN,2_05:35_Weekday_82661,Boise & Law NWC,NaN,7/1/2019 12:00:00 AM,7/1/2019 5:35:00 AM,7/1/2019 6:06:00 AM,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,Broadway,NaN,2_05:35_Weekday_82661,Broadway & Beacon NEC,NaN,7/1/2019 12:00:00 AM,7/1/2019 5:35:00 AM,7/1/2019 6:06:00 AM,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,Broadway,NaN,2_05:35_Weekday_82661,Main Street Station,NaN,7/1/2019 12:00:00 AM,7/1/2019 5:35:00 AM,7/1/2019 6:06:00 AM,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
df['Count']=1
df['Early']=df['MinutesEarly']>0
df['On Time']=(df['MinutesEarly']==0)&(df['MinutesLate']==0)
df['Late']=df['MinutesLate']>0
df['Hour']=df['ScheduledStopTime'].dt.hour
df.head(50)

,VehicleName,RunName,MasterRouteName,SubRouteName,TripName,StopName,DriverName,TripDate,ScheduledTripStart,ScheduledTripEnd,...,route_type,route_url,route_color,route_text_color,Route Name,Count,Early,On Time,Late,Hour
0,NaN,NaN,Broadway,NaN,2_05:35_Weekday_82661,Boise Airport,NaN,7/1/2019 12:00:00 AM,7/1/2019 5:35:00 AM,7/1/2019 6:06:00 AM,...,NaN,NaN,NaN,NaN,NaN,1,False,True,False,5
1,NaN,NaN,Broadway,NaN,2_05:35_Weekday_82661,Federal Way @ Fred Meyer SWC,NaN,7/1/2019 12:00:00 AM,7/1/2019 5:35:00 AM,7/1/2019 6:06:00 AM,...,NaN,NaN,NaN,NaN,NaN,1,False,True,False,5
2,NaN,NaN,Broadway,NaN,2_05:35_Weekday_82661,Boise & Law NWC,NaN,7/1/2019 12:00:00 AM,7/1/2019 5:35:00 AM,7/1/2019 6:06:00 AM,...,NaN,NaN,NaN,NaN,NaN,1,False,True,False,5
3,NaN,NaN,Broadway,NaN,2_05:35_Weekday_82661,Broadway & Beacon NEC,NaN,7/1/2019 12:00:00 AM,7/1/2019 5:35:00 AM,7/1/2019 6:06:00 AM,...,NaN,NaN,NaN,NaN,NaN,1,False,True,False,5
4,NaN,NaN,Broadway,NaN,2_05:35_Weekday_82661,Main Street Station,NaN,7/1/2019 12:00:00 AM,7/1/2019 5:35:00 AM,7/1/2019 6:06:00 AM,...,NaN,NaN,NaN,NaN,NaN,1,False,False,True,6
5,NaN,NaN,Broadway,NaN,2_06:05_Weekday_82662,Boise Airport,NaN,7/1/2019 12:00:00 AM,7/1/2019 6:05:00 AM,7/1/2019 6:36:00 AM,...,NaN,NaN,NaN,NaN,NaN,1,False,True,False,6
6,NaN,NaN,Broadway,NaN,2_06:05_Weekday_82662,Federal Way @ Fred Meyer SWC,NaN,7/1/2019 12:00:00 AM,7/1/2019 6:05:00 AM,7/1/2019 6:36:00 AM,...,NaN,NaN,NaN,NaN,NaN,1,False,True,False,6
7,NaN,NaN,Broadway,NaN,2_06:05_Weekday_82662,Boise & Law NWC,NaN,7/1/2019 12:00:00 AM,7/1/2019 6:05:00 AM,7/1/2019 6:36:00 AM,...,NaN,NaN,NaN,NaN,NaN,1,False,True,False,6
8,NaN,NaN,Broadway,NaN,2_06:05_Weekday_82662,Broadway & Beacon NEC,NaN,7/1/2019 12:00:00 AM,7/1/2019 6:05:00 AM,7/1/2019 6:36:00 AM,...,NaN,NaN,NaN,NaN,NaN,1,False,True,False,6
9,NaN,NaN,Broadway,NaN,2_06:05_Weekday_82662,Main Street Station,NaN,7/1/2019 12:00:00 AM,7/1/2019 6:05:00 AM,7/1/2019 6:36:00 AM,...,NaN,NaN,NaN,NaN,NaN,1,False,False,True,6


In [16]:
outdf=pd.DataFrame(columns=['Early','On Time','Late','Count'])
group_cols=['MasterRouteName','TripName','StopName','StopOrder','Hour']
outdf['Early']=df.groupby(group_cols)['Early'].sum()
outdf['On Time']=df.groupby(group_cols)['On Time'].sum()
outdf['Late']=df.groupby(group_cols)['Late'].sum()
outdf['Count']=df.groupby(group_cols)['Count'].sum()
outdf.reset_index(inplace=True)
outdf.sort_values(['MasterRouteName','TripName','StopOrder','StopName','Hour'],inplace=True)
outdf=outdf.groupby(['MasterRouteName','StopName','Hour'],as_index=False,sort=False).sum()
outdf['Early']=outdf['Early']/outdf['Count']
outdf['On Time']=outdf['On Time']/outdf['Count']
outdf['Late']=outdf['Late']/outdf['Count']
# outdf=outdf.merge(feed,left_on=['MasterRouteName','StopName'],right_on=['route_long_name','stop_name'],how='left')
outdf=outdf[['MasterRouteName','StopName','Hour','Early','On Time','Late']]
outdf.rename({'StopName':'Stop Name'},inplace=True,axis=1)
outdf

,MasterRouteName,Stop Name,Hour,Early,On Time,Late
0,BSU Express,CWI Main Campus,9,0.000000,0.611111,0.388889
1,BSU Express,9th & River SWC,9,0.000000,0.000000,1.000000
2,BSU Express,University & Joyce SEC,9,0.000000,0.388889,0.611111
3,BSU Express,Front & 3rd NEC,9,0.000000,0.222222,0.777778
4,BSU Express,Front & 9th NEC,9,0.000000,0.222222,0.777778
5,BSU Express,CWI Main Campus,10,0.000000,1.000000,0.000000
6,BSU Express,9th & River SWC,10,0.000000,1.000000,0.000000
7,BSU Express,University & Joyce SEC,10,0.000000,1.000000,0.000000
8,BSU Express,Front & 3rd NEC,10,0.000000,0.888889,0.111111
9,BSU Express,Front & 9th NEC,10,0.000000,1.000000,0.000000


In [10]:
outdf.to_excel(r"\\VRIDE-FS2\Development\Planning - New File Structure\Performance\FY19 OTP\ToD_7_1_7_13_19.xlsx",index=False)